# Proyecto Final <br>
# Estado de Servicios en Zonas no Interconectadas de Colombia <br>

### Por: Juan Diego Carvajal D, Daniel, César

## Datos

La base de datos seleccionada es una base de datos que recoge variables de telemetría tomadas desde el 21 de Mayo de 2025 hasta el 27 de Mayo de 2025 en las zonas no interconectadas de Colombia en las localidades donde es posible realizar seguimiento. <br>

Para realizar la visualización de los datos en Python es necesario importar las siguientes librerías:

In [1]:
import numpy as np
import pandas as pd
import datetime

Para una visualización inicial de los datos se transforman los datos de CSV a DataFrame

In [7]:
energias_original = pd.read_csv('Estado_de_la_prestaci_n_del_servicio_de_energ_a_en_Zonas_No_Interconectadas_20250702.csv')
energias_original.shape

(4660, 14)

Encontrando las siguientes variables:

In [ ]:
energias_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4660 entries, 0 to 4659
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID DEPATAMENTO            4660 non-null   int64  
 1   DEPARTAMENTO              4660 non-null   object 
 2   ID MUNICIPIO              4660 non-null   int64  
 3   MUNICIPIO                 4660 non-null   object 
 4   ID LOCALIDAD              4660 non-null   int64  
 5   LOCALIDAD                 4660 non-null   object 
 6   AÑO SERVICIO              4660 non-null   int64  
 7   MES SERVICIO              4660 non-null   int64  
 8   ENERGÍA ACTIVA            4660 non-null   int64  
 9   ENERGÍA REACTIVA          4660 non-null   float64
 10  POTENCIA MÁXIMA           4660 non-null   float64
 11  DÍA DE DEMANDA MÁXIMA     4659 non-null   object 
 12  FECHA DE DEMANDA MÁXIMA   4660 non-null   object 
 13  PROMEDIO DIARIO EN HORAS  4660 non-null   float64
dtypes: float

In [13]:
energias = energias_original
energias['FECHA DE DEMANDA MÁXIMA'] = pd.to_datetime(energias['FECHA DE DEMANDA MÁXIMA'], format='%m/%d/%Y %I:%M:%S %p')
energias = energias.drop(columns=['AÑO SERVICIO', 'MES SERVICIO', 'DÍA DE DEMANDA MÁXIMA', 'LOCALIDAD', 'DEPARTAMENTO', 'MUNICIPIO'])
energias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4660 entries, 0 to 4659
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   ID DEPATAMENTO            4660 non-null   int64         
 1   ID MUNICIPIO              4660 non-null   int64         
 2   ID LOCALIDAD              4660 non-null   int64         
 3   ENERGÍA ACTIVA            4660 non-null   int64         
 4   ENERGÍA REACTIVA          4660 non-null   float64       
 5   POTENCIA MÁXIMA           4660 non-null   float64       
 6   FECHA DE DEMANDA MÁXIMA   4660 non-null   datetime64[ns]
 7   PROMEDIO DIARIO EN HORAS  4660 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(4)
memory usage: 291.4 KB


La energía activa es aquella que se transforma en trabajo útil, la energía reactiva no se consume directamente como trabajo útil, la potencia máxima es la mayor cantidad de energía respecto al tiempo que el sistema eléctrico pudo manejar, la fecha de demanda máxima es la fecha y hora que hubo mayor demanda y el promedio diario en horas son las horas que la localidad consumió energía en la fecha especificada. <br>

Para iniciar con la limpieza de datos se decide consultar una nueva base de datos que brinde información sobre las localidades, para esto se usa la base de datos de la división política administrativa DIVIPOLA.

In [5]:
divapola = pd.read_csv('Geoportal del DANE - Codificación Divipola.csv')
divapola.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8420 entries, 0 to 8419
Data columns (total 12 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Código Departamento                       8420 non-null   int64  
 1   Código Municipio                          8420 non-null   int64  
 2   Código Centro Poblado                     8420 non-null   int64  
 3   Nombre Departamento                       8420 non-null   object 
 4   Nombre Municipio                          8420 non-null   object 
 5   Nombre Centro Poblado                     8420 non-null   object 
 6   Tipo Centro Poblado                       8420 non-null   object 
 7   Longitud                                  7664 non-null   float64
 8   Latitud                                   7664 non-null   float64
 9   Nombre Distrito                           494 non-null    object 
 10  Municipio/Áreas No Municipalizadas (

En la base de datos DIVIPOLA encontramos información sobre el departamento, el municipio y el centro poblado.

Además, se cree pertinente incluir información sobre el Censo.

In [14]:
censo = pd.read_excel('CNPV-2018-NBI-CENTROS-POBLADOS.xlsx', skiprows=8)
censo = censo.rename(columns={
    'Unnamed: 0': 'Código de departamento',
    'Unnamed: 1': 'Departamento',
    'Unnamed: 2': 'Código de municipio',
    'Unnamed: 3': 'Municipio',
    'Unnamed: 4': 'Clase',
    'Unnamed: 5': 'Código de centro poblado',
    'Unnamed: 6': 'Centro Poblado',
    'Unnamed: 14': 'Total personas en hogares particulares'
})
censo = censo.drop([8558, 8559])
censo['Total personas en hogares particulares'] = censo['Total personas en hogares particulares'].astype('Int64')
censo['Código de municipio'] = censo['Código de municipio'].astype(int)
censo['Código de centro poblado'] = censo['Código de centro poblado'].astype(int)

censo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8558 entries, 0 to 8557
Data columns (total 15 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Código de departamento                  8558 non-null   object 
 1   Departamento                            8558 non-null   object 
 2   Código de municipio                     8558 non-null   int64  
 3   Municipio                               8558 non-null   object 
 4   Clase                                   8558 non-null   object 
 5   Código de centro poblado                8558 non-null   int64  
 6   Centro Poblado                          8558 non-null   object 
 7   Personas en NBI (%)                     8558 non-null   float64
 8    Personas en miseria (%)                8558 non-null   float64
 9   Componente vivienda (%)                 8558 non-null   float64
 10  Componente Servicios (%)                8558 non-null   floa

Estos datos son exportados a PostreSQL

## SQL

Se crean las tablas departamentos, muninicipios, centros poblados como parte del esquema DIVIPOLA; las tablas cnpv y nbi como parte del equipos Censo y, por último, las tabla de servicios energéticos como parte del esquema de Energías.

In [ ]:
-- CREATE DATABASE energia_zonas_no_interconectadas_col;
-- USE energia_zonas_no_interconectadas_col;
-- SET SQL_SAFE_UPDATES = 0;

CREATE TABLE divapola.tipos_centro_poblado(
	id_tipo_centro_poblado SERIAL PRIMARY KEY,
    tipo_centro_poblado TEXT
);

-- Se añade "Rural disperso" que no está en el divapola pero sí en el censo.

INSERT INTO divapola.tipos_centro_poblado (tipo_centro_poblado) VALUES 
('CABECERA MUNICIPAL'), ('CENTRO POBLADO'), ('RURAL DISPERSO');

ALTER TABLE divapola.tipos_centro_poblado RENAME TO clases;
ALTER TABLE divapola.clases RENAME COLUMN id_tipo_centro_poblado to id_clase;
ALTER TABLE divapola.clases RENAME COLUMN tipo_centro_poblado to clase;


CREATE TABLE divapola.tipos_municipio(
	id_tipo_municipio SERIAL PRIMARY KEY,
    tipo_municipio TEXT
);
INSERT INTO divapola.tipos_municipio (tipo_municipio) VALUES
('MUNICIPIO'), ('ISLA'), ('AREA NO MUNICIPALIZADA');

CREATE TABLE divapola.departamentos (
	id_departamento SERIAL PRIMARY KEY,
    departamento TEXT
);

CREATE TABLE divapola.distritos (
	id_distrito SERIAL PRIMARY KEY,
	distrito TEXT
); 

CREATE TABLE divapola.areas_metropolitanas (
	id_area_metropolitana SERIAL PRIMARY KEY,
	area_metropolitana TEXT
);

CREATE TABLE divapola.municipios(
	id_municipio SERIAL PRIMARY KEY,
	municipio TEXT, 
	id_departamento INT, FOREIGN KEY (id_departamento) REFERENCES divapola.departamentos(id_departamento),
	id_distrito INT, FOREIGN KEY (id_distrito) REFERENCES divapola.distritos (id_distrito),
	id_tipo_municipio INT, FOREIGN KEY (id_tipo_municipio) REFERENCES divapola.tipos_municipio (id_tipo_municipio),
	id_area_metropolitana INT, FOREIGN KEY (id_area_metropolitana) REFERENCES divapola.areas_metropolitanas (id_area_metropolitana)
);


CREATE TABLE divapola.centros_poblados(
	id_centro_poblado SERIAL PRIMARY KEY,
	centro_poblado TEXT, 
	id_municipio INT, FOREIGN KEY (id_municipio) REFERENCES divapola.municipios (id_municipio),
	longitud DOUBLE PRECISION,
	latitud DOUBLE PRECISION,
	id_tipo_centro_poblado INT, FOREIGN KEY (id_tipo_centro_poblado) REFERENCES divapola.tipos_centro_poblado(id_tipo_centro_poblado)
);

ALTER TABLE divapola.centros_poblados RENAME COLUMN id_tipo_centro_poblado TO id_clase;


CREATE TABLE energias.servicios_centros_poblados(
	id_servicio SERIAL PRIMARY KEY,
	id_centro_poblado INT, FOREIGN KEY (id_centro_poblado) REFERENCES divapola.centros_poblados(id_centro_poblado),
	energia_activa DOUBLE PRECISION,
	energia_reactiva DOUBLE PRECISION,
	potencia_maxima DOUBLE PRECISION,
	fecha_demanda_maxima TIMESTAMP,
	promedio_diario_horas DOUBLE PRECISION
);

CREATE TABLE cnpv_nbi.cnpv_nbi(
	id_cnpv_nbi SERIAL PRIMARY KEY,
	id_centro_poblado INT, FOREIGN KEY (id_centro_poblado) REFERENCES divapola.centros_poblados(id_centro_poblado),
	personas_nbi DOUBLE PRECISION,
	personas_miseria DOUBLE PRECISION,
	vivienda DOUBLE PRECISION,
	servicios DOUBLE PRECISION,
	hacinamiento DOUBLE PRECISION,
	inasistencia DOUBLE PRECISION,
	dependencia_economica DOUBLE PRECISION,
	personas_hogares_particulares INT
);

-- DATOS QUE ESTÁN EN LA TABLA DE ENERGÍAS PERO NO EN EL DIVAPOLA
-- Algunos se hicieron desde la tabla y otros desde script

SELECT * FROM divapola.centros_poblados where centro_poblado is null order by id_centro_poblado;

UPDATE divapola.centros_poblados SET centro_poblado = 'LA CONCHA - CONCEPCIÓN NAYA' WHERE id_centro_poblado = 19418007;
UPDATE divapola.centros_poblados SET centro_poblado = 'RIO MAYA - DOS QUEBRADAS' WHERE id_centro_poblado = 19418011;
UPDATE divapola.centros_poblados SET centro_poblado = 'MARCIAL' WHERE id_centro_poblado = 27615912;
UPDATE divapola.centros_poblados SET centro_poblado = 'GALVEZ ' WHERE id_centro_poblado = 76109930;

SELECT * FROM energias.servicios_centros_poblados;


-- DATOS QUE ESTÁN EN LA TABLA DE CNPV/NBI PERO NO EN EL DIVAPOLA

INSERT INTO divapola.municipios (id_municipio, municipio, id_departamento) VALUES (94663, 'MAPIRIPANA', 94);
SELECT * FROM divapola.centros_poblados;    

![Texto alternativo](ERD_ProyectoFinal.pgerd.png)

Se realiza la consulta que generará la vista para usar como base en Streamlit

In [ ]:
CREATE OR REPLACE VIEW energias.servicios_detalle AS
SELECT 
	d.id_departamento AS "Código Departamento",
	d.departamento AS "Departamento",
	m.id_municipio AS "Código Municipio",
	m.municipio AS "Municipio",
	c.id_centro_poblado AS "Código Centro Poblado",
	c.centro_poblado AS "Centro Poblado",
	c.latitud AS "Latitud",
	c.longitud AS "Longitud",
	e.energia_activa AS "Energía Activa [kWh]",
	e.energia_reactiva AS "Energía Reactiva [kVArh]",
	(e.energia_activa/NULLIF(SQRT(POWER(e.energia_activa, 2)+POWER(e.energia_reactiva,2)), 0)) AS "Factor de Potencia",
	e.potencia_maxima AS "Potencia Máxima [kW]",
	e.fecha_demanda_maxima AS "Fecha Demanda Máxima",
	e.promedio_diario_horas AS "Promedio Diario [h]",
	n.personas_hogares_particulares AS "Total Personas en Hogares Particulares",
	n.personas_nbi AS "Personas en NBI [%]",
	n.servicios AS "Componente Servicios [%]"
FROM energias.servicios_centros_poblados e
LEFT JOIN divapola.centros_poblados c ON e.id_centro_poblado = c.id_centro_poblado
LEFT JOIN divapola.municipios m ON c.id_municipio = m.id_municipio
LEFT JOIN divapola.departamentos d ON m.id_departamento = d.id_departamento
LEFT JOIN cnpv_nbi.cnpv_nbi n ON c.id_centro_poblado = n.id_centro_poblado;

## Streamlit

#### Macros

In [ ]:
import streamlit as st
from sqlalchemy import create_engine, text
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math
from scipy.stats import gaussian_kde
import pydeck as pdk
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score